In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### 1. Import Data

In [3]:
df_raw = pd.read_csv('data/Sample_Data.csv')

/Users/redstockings/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (10,13,14,33) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df_raw.head()

,Unnamed: 0,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,...,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,PotentialFraud
0,0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,...,1,1,2,1,1,36000,3204,60,70,Yes
1,1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,...,1,1,2,1,1,36000,3204,60,70,No
2,2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,...,1,1,2,1,1,36000,3204,60,70,No
3,3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,...,1,2,2,1,1,5000,1068,250,320,No
4,4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,...,2,1,2,2,2,21260,2136,120,100,No


In [5]:
# remove "unnamed" column
df_raw = df_raw.iloc[:,1:]

In [6]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558211 entries, 0 to 558210
Data columns (total 56 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   BeneID                           558211 non-null  object 
 1   ClaimID                          558211 non-null  object 
 2   ClaimStartDt                     558211 non-null  object 
 3   ClaimEndDt                       558211 non-null  object 
 4   Provider                         558211 non-null  object 
 5   InscClaimAmtReimbursed           558211 non-null  int64  
 6   AttendingPhysician               556703 non-null  object 
 7   OperatingPhysician               114447 non-null  object 
 8   OtherPhysician                   199736 non-null  object 
 9   AdmissionDt                      40474 non-null   object 
 10  ClmAdmitDiagnosisCode            145899 non-null  object 
 11  DeductibleAmtPaid                557312 non-null  float64
 12  Di

### 2. Aggregate features grouped by Providers

#### 2.1 BeneID

In [9]:
ClaimCount = df_raw.groupby(['Provider']).ClaimID.count()

In [10]:
uniqBeneCount = df_raw.groupby(['Provider']).BeneID.nunique()

In [11]:
ClaimCount - uniqBeneCount  # There are some duplicated beneficiaries

Provider
PRV51001      1
PRV51003     15
PRV51004     11
PRV51005    670
PRV51007     14
           ... 
PRV57759      4
PRV57760     13
PRV57761     15
PRV57762      0
PRV57763     48
Length: 5410, dtype: int64

In [12]:
ClmperBene = ClaimCount/uniqBeneCount
ClmperBene

Provider
PRV51001    1.041667
PRV51003    1.128205
PRV51004    1.079710
PRV51005    2.353535
PRV51007    1.241379
              ...   
PRV57759    1.166667
PRV57760    2.444444
PRV57761    1.223881
PRV57762    1.000000
PRV57763    1.685714
Length: 5410, dtype: float64

#### 2.2 InscClaimAmtReimbursed

In [13]:
TotalClmAmtReim = df_raw.groupby(['Provider']).InscClaimAmtReimbursed.sum()

In [14]:
MeanClmAmtReim = df_raw.groupby(['Provider']).InscClaimAmtReimbursed.mean()

In [15]:
MaxClmAmtReim = df_raw.groupby(['Provider']).InscClaimAmtReimbursed.max()
MaxClmAmtReim

Provider
PRV51001    42000
PRV51003    57000
PRV51004     3300
PRV51005     4080
PRV51007    10000
            ...  
PRV57759     3300
PRV57760     1100
PRV57761     2200
PRV57762     1900
PRV57763     3300
Name: InscClaimAmtReimbursed, Length: 5410, dtype: int64

#### 2.3 AttendingPhysician

In [16]:
uniqAtPhysnCount= df_raw.groupby(['Provider']).AttendingPhysician.nunique()

In [17]:
ClmperAtPhysn = ClaimCount/uniqAtPhysnCount
ClmperAtPhysn

Provider
PRV51001      1.785714
PRV51003      3.000000
PRV51004      3.921053
PRV51005    194.166667
PRV51007      7.200000
               ...    
PRV57759     28.000000
PRV57760      7.333333
PRV57761     41.000000
PRV57762      1.000000
PRV57763    118.000000
Length: 5410, dtype: float64

#### 2.4 OperatingPhysician

In [18]:
uniqOPPhysnCount = df_raw.groupby(['Provider']).OperatingPhysician.nunique()

In [19]:
ClmperOPPhysn = ClaimCount/uniqOPPhysnCount
ClmperOPPhysn

Provider
PRV51001      5.000000
PRV51003     18.857143
PRV51004      7.842105
PRV51005    194.166667
PRV51007     14.400000
               ...    
PRV57759     28.000000
PRV57760     11.000000
PRV57761     16.400000
PRV57762           inf
PRV57763    118.000000
Length: 5410, dtype: float64

#### 2.5 OtherPhysician

In [20]:
uniqOTPhysnCount = df_raw.groupby(['Provider']).OtherPhysician.nunique()

In [21]:
ClmperOTPhysn = ClaimCount/uniqOTPhysnCount
ClmperOTPhysn

Provider
PRV51001      3.125000
PRV51003      6.000000
PRV51004      5.730769
PRV51005    291.250000
PRV51007     10.285714
               ...    
PRV57759     28.000000
PRV57760     11.000000
PRV57761     41.000000
PRV57762           inf
PRV57763    118.000000
Length: 5410, dtype: float64

#### 2.6 AdmissinoDt and DischargeDt

In [22]:
HospitalStay = pd.to_datetime(df_raw.DischargeDt) - pd.to_datetime(df_raw.AdmissionDt)

In [23]:
type(df_raw.DischargeDt.iloc[0])

str

In [24]:
HospitalStay  # Timedelta; Only inpatient claims have HospitalStay

0         6 days
1         2 days
2         3 days
3         8 days
4        17 days
           ...  
558206       NaT
558207       NaT
558208       NaT
558209       NaT
558210       NaT
Length: 558211, dtype: timedelta64[ns]

In [25]:
# convert timedelta to float number (dt.days) and NaT to 0
HospitalStay = HospitalStay.dt.days.apply(lambda x: 0 if np.isnan(x) else x)  

In [26]:
HospitalStay.index = df_raw.Provider

In [27]:
HospitalStay

Provider
PRV55912     6.0
PRV55907     2.0
PRV56046     3.0
PRV52405     8.0
PRV56614    17.0
            ... 
PRV53699     0.0
PRV53702     0.0
PRV53676     0.0
PRV53689     0.0
PRV53689     0.0
Length: 558211, dtype: float64

In [28]:
HospitalStay_mean = HospitalStay.groupby(['Provider']).mean()
HospitalStay_max = HospitalStay.groupby(['Provider']).max()

#### 2.7 In_Out

In [29]:
InClmsPct = df_raw.groupby(['Provider']).In_Out.sum()/ClaimCount  # 1 means in patient clms

In [30]:
InClmsPct  # Number of inpatient claims

Provider
PRV51001    0.200000
PRV51003    0.469697
PRV51004    0.000000
PRV51005    0.000000
PRV51007    0.041667
              ...   
PRV57759    0.000000
PRV57760    0.000000
PRV57761    0.000000
PRV57762    0.000000
PRV57763    0.000000
Length: 5410, dtype: float64

### Export Data Table for Team

In [31]:
data = {'uniqBeneCount':uniqBeneCount,
        'ClmperBene':ClmperBene, 
        'MeanInscClaimAmtReimbursed':MeanClmAmtReim,
        'MaxInscClaimAmtReimbursed':MaxClmAmtReim,
        'uniqAtPhysnCount':uniqAtPhysnCount,
        'ClmperAtPhysn':ClmperAtPhysn,
        'uniqOPPhysnCount':uniqOPPhysnCount,
        'ClmperOPPhysn':ClmperOPPhysn,
        'uniqOTPhysnCount':uniqOTPhysnCount,
        'ClmperOTPhysn':ClmperOTPhysn,
        'InClmsPct':InClmsPct}
df_provider_mz = pd.DataFrame(data = data)

In [32]:
df_provider_mz.head()

,uniqBeneCount,ClmperBene,MeanInscClaimAmtReimbursed,MaxInscClaimAmtReimbursed,uniqAtPhysnCount,ClmperAtPhysn,uniqOPPhysnCount,ClmperOPPhysn,uniqOTPhysnCount,ClmperOTPhysn,InClmsPct
Provider,,,,,,,,,,,
PRV51001,24,1.041667,4185.600000,42000,14,1.785714,5,5.000000,8,3.125000,0.200000
PRV51003,117,1.128205,4588.409091,57000,44,3.000000,7,18.857143,22,6.000000,0.469697
PRV51004,138,1.079710,350.134228,3300,38,3.921053,19,7.842105,26,5.730769,0.000000
PRV51005,495,2.353535,241.124464,4080,6,194.166667,6,194.166667,4,291.250000,0.000000
PRV51007,58,1.241379,468.194444,10000,10,7.200000,5,14.400000,7,10.285714,0.041667


In [33]:
df_provider_mz.to_csv("df_provider_mz.csv")